In [1]:
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn import metrics
from contextlib import redirect_stdout
import scipy.sparse as sparse
import lightgbm as lgb
import pandas as pd
import numpy as np
import json
import tqdm
import os
import gc
import sys
sys.path.append('../../../code/pipeline/')
sys.path.append('../../../code/utils/')
sys.path.append('../../../code/')
import data_pipeline as dp
import data_utils as du
import perf_utils as pu
import eval_utils as eu
import io_utils as iu
import config

In [2]:
# load multiple data and stack them together
input_folder = os.path.join(config.DATA_DIR, "input_final")
dm = dp.DataManager(input_folder)

lc_v1 = dm.build_data('aggregate.listCount_v1')
lc_v2 = dm.build_data('aggregate.listCount_v2')
clk_v1 = dm.build_data('clickStats.crossWordCount_v1')
clk_v2 = dm.build_data('clickStats.crossWordCount_v2')
stack_cross_v1 = dm.build_data('stacking.lr.crossWordCount_v1')
stack_cross_v1_all = dm.build_data('stacking.lr.crossWordCount_v1_all')
stack_cross_v2 = dm.build_data('stacking.lr.crossWordCount_v2')
stack_cross_v3 = dm.build_data('stacking.lr.crossWordCount_v3')
stack_cross_v3_all = dm.build_data('stacking.lr.crossWordCount_v3_all')
stack_emb = dm.build_data('stacking.lr.wordEmbedding')
stack_tfidf = dm.build_data('stacking.lr.wordTfIdf')
raw = dm.build_data('raw.wordCount')

In [3]:
union_loader = dp.DataUnion(lc_v1, lc_v2, 
                            clk_v1, clk_v2,
                            stack_cross_v1, stack_cross_v1_all, 
                            stack_cross_v2, 
                            stack_cross_v3, stack_cross_v3_all, 
                            stack_emb, 
                            stack_tfidf, 
                            raw)

In [8]:
# with pu.profiler("loading training data"):
#     cols_train, X_tv = union_loader.load("train")
#     X_tv = sparse.csr_matrix(X_tv)
#     gc.collect()
# print("Train Data Shape: {}".format(X_tv.shape))
# print("Train Column Numbers: {}".format(len(cols_train)))

df_train = du.load_raw_data("train")
y = df_train['label'].values.copy()
y = (y + 1) / 2  # -1, 1 -> 0, 1

In [9]:
n_splits = 5
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=2018)  # for reproducibility
split_indices = [(train_index, valid_index) for train_index, valid_index in sss.split(df_train, y)]
# n_splits = 5  # use 3 instead of 5 to save time
# skf = StratifiedKFold(n_splits=n_splits, random_state=2018)
# split_indices = [(train_index, valid_index) for train_index, valid_index in skf.split(df_train, y)]

aids = df_train['aid'].values
with pu.profiler("splitting train/valid set"):
    train_index, valid_index = split_indices[0]
    X_train, X_valid = X_tv[train_index, :], X_tv[valid_index, :]
    y_train, y_valid = y[train_index], y[valid_index]
    aids_train, aids_valid = aids[train_index], aids[valid_index]
    assert X_train.shape[0] + X_valid.shape[0] == X_tv.shape[0]
    
    del X_tv
    gc.collect()
    
print("Training Set Size: {}".format(X_train.shape))
print("Validation Set Size: {}".format(X_valid.shape))

[13:30:44] Finish splitting train/valid set. △M: -10.52GB. △T: 4.0 minutes.
Training Set Size: (7039051, 419818)
Validation Set Size: (1759763, 419818)


In [10]:
with pu.profiler("preparing LightGBM data"):
    lgb_train = lgb.Dataset(X_train.astype(np.float32), y_train)
    del X_train  # to save memory
    gc.collect()
    
    lgb_valid = lgb.Dataset(X_valid.astype(np.float32), y_valid)
    gc.collect()

[05:30:17] Finish preparing LightGBM data. △M: +8.06GB. △T: 1.4 minutes.


In [11]:
# v1: 
version_name = "final/subconscious_v1"
log_folder = os.path.join(config.LOG_DIR, 'lgbm/{}'.format(version_name))
os.makedirs(log_folder, exist_ok=True)

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': 6,
    'num_leaves': 64,
    'learning_rate': 0.1,
    'feature_fraction': 0.001,
    'bagging_fraction': 0.8,
    'verbose': 0
}
num_rounds = 800000

log_file = 'params.json'
log_path = os.path.join(log_folder, log_file)
with open(log_path, 'w') as f:
    json.dump(params, f, indent=4)

log_file = 'log.txt'
log_path = os.path.join(log_folder, log_file)
eval_auc = eu.build_lightgbm_online_auc_eval(aids_train, aids_valid)
with iu.DuplicatedLogger(log_path):
    lgbm = lgb.train(params,
                     lgb_train,
                     num_boost_round=num_rounds,
                     valid_sets=[lgb_train, lgb_valid], 
                     valid_names=['train', 'valid'],
                     feval = eval_auc,
                     verbose_eval=50,
                     early_stopping_rounds=50)

ValueError: continuous format is not supported

In [7]:
log_file = 'feature_importance.csv'
log_path = os.path.join(log_folder, log_file)

df_feature_importance = pd.DataFrame({"feature": cols_train, "importance": lgbm.feature_importance()})
df_feature_importance = df_feature_importance.sort_values("importance", ascending=False)
df_feature_importance.to_csv(log_path, index=False)

In [9]:
df_valid = df_train.iloc[valid_index]
proba_valid = lgbm.predict(X_valid.astype(np.float32))
df_score = eu.online_auc(df_valid['aid'], y_valid, proba_valid, ret_verbose=True)

online_auc = df_score['auc'].mean()
simple_auc = metrics.roc_auc_score(y_valid, proba_valid)
print("Online AUC: {:.6f}".format(online_auc))
print("Simple AUC: {:.6f}".format(simple_auc))

log_file = 'online_auc.csv'
log_path = os.path.join(log_folder, log_file)
df_score.rename(columns={'selector': 'aid'}, inplace=True)
df_score = df_score[['aid', 'auc']]  # sort columns
df_score = df_score.sort_values("auc", ascending=False)
df_score.to_csv(log_path, index=False)

Online AUC: 0.725787
Simple AUC: 0.731557


In [10]:
with pu.profiler("cleaning memory"):
    del lgb_train
    del lgb_valid
    # del X_train
    del X_valid
    gc.collect()

[03:40:42] Finish cleaning memory. △M: -5.66GB. △T: 1.0 seconds.


In [22]:
with pu.profiler("loading testing data"):
    cols_test, X_test = union_loader.load("test2")
    cols_train_set = set(cols_train)
    mask = [i for i, col in enumerate(cols_test) if col in cols_train_set]
    X_test = sparse.csr_matrix(X_test[:, mask])
    gc.collect()
print("Test Data Shape: {}".format(X_test.shape))
print("Test Column Numbers: {}".format(len(cols_test)))

df_test = du.load_raw_data("test2")
X_test = X_test.astype(np.float32)

with pu.profiler("making prediction on testing set"):
    proba_test = lgbm.predict(X_test)
    assert len(proba_test.shape) == 1
    assert proba_test.shape[0] == df_test.shape[0]
    
subm_folder = '../../../subm/lgbm/{}'.format(version_name)
subm_file = 'submission.csv'
subm_path = os.path.join(subm_folder, subm_file)
os.makedirs(subm_folder, exist_ok=True)

subm = df_test.copy()
subm["score"] = proba_test
subm.to_csv(subm_path, index=False)

[04:38:37] Finish loading testing data. △M: -661.99MB. △T: 8.3 seconds.
Test Data Shape: (2265879, 419701)
Test Column Numbers: 419702
[04:38:59] Finish making prediction on testing set. △M: +0B. △T: 19.3 seconds.


In [21]:
len(mask)

419701